In [1]:
# 작성자 : 이민우
# 작성일 : 20181101
# 프로그램설명 : 머신러닝을 이용한 가격예측
import os 
import tarfile
import urllib.request
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import warnings

warnings.filterwarnings('ignore')
TEST_PATH = os.path.join("datasets","lab06") # 경로 지정

FirststFlrSF_ix,SecondFlrSF_ix,BsmtFullBath_ix,BsmtHalfBath_ix,FullBath_ix,HalfBath_ix = 13,14,17,18,19,20
#변환기
class CombinedAttributesAdder(BaseEstimator, TransformerMixin): # BaseEstimator 와 TransformerMixin 상속받을시 get_param , set_param
    #fit_trnsform 함수 자동 구현 
    def __init__(self, add_TotalSquareFeet = True):
        self.add_TotalSquareFeet = add_TotalSquareFeet
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        TotalBathRoom = X[:, BsmtFullBath_ix] + X[:, BsmtHalfBath_ix]+ X[:,FullBath_ix]+ X[:, HalfBath_ix] # 총 욕조수 
        
        if self.add_TotalSquareFeet:           
            TotalSquareFeet = X[:, FirststFlrSF_ix] + X[:,SecondFlrSF_ix] # 1,2 층 넓이의 합 
            return np.c_[X,TotalSquareFeet,TotalBathRoom]
        else:
            return np.c_[X,TotalBathRoom]

        
class DataFrameSelector(BaseEstimator, TransformerMixin): # 데이터프레임을 다룰수 없어 numpy 배열로 전환하는 클래스
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names 
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values # 주어진 attribute_name에 해당하는 값을 반환 
    
    
def load_housing_data(filename, test_path = TEST_PATH): # 주어진 filename 을 읽는 함수
    csv_path = os.path.join(test_path,filename)
    return pd.read_csv(csv_path)
   
def display_score(sco): # 보기 쉽게 점수를 출력 
    print("점수 : ", sco)
    print("평균 : ", sco.mean())
    print("표준편차 : ", sco.std())
    
if __name__ == "__main__":
    # 주어진 링크에서 all.zip 을 다운받아 압축을 푼 후 진행하였습니다.
    
    train_set = load_housing_data("train.csv")
    test_set = load_housing_data("test.csv")
    sample_set = load_housing_data("sample_submission.csv")
    
    train_set["SalePrice"].hist()
    plt.show()
    #데어터 셋 내용보기.
    
   # print(train_set)
    print(train_set.info())
    print(train_set.head())
    
    train = train_set.copy()
    
    train_set.hist(bins = 50, figsize = (20,15))
    plt.show()
    
    # 상관관계
    train_copy = train_set.copy()
    train_copy["TotalSquareFeet"] = train_copy['2ndFlrSF']+train_copy['1stFlrSF']
    train_copy["TotalBathRoom"] = train_copy['FullBath']+train_copy['HalfBath']+train_copy['BsmtFullBath']+train_copy['BsmtHalfBath']
  
    corr_matrix = train_copy.corr()
    print(corr_matrix["SalePrice"].sort_values(ascending = False))
    #TotalSquareFeet 특성은 2ndFlrSF 특성이나 HalfBath 특성보다 높은 상관관게를 갖는다.  1,2 층 넓이의 합 
    #otalBathRoom 특성은 FullBath 특성이나 BsmtFullBath 특성보다 높은 상관관계를 갖는다. 총 욕조 수 
 
    att = ['SalePrice', 'OverallQual', 'TotalSquareFeet','GrLivArea']
    scatter_matrix(train_copy[att], figsize = (12,8))
    plt.show()
    
    # fillna 함수를 통한 중간 값 계산 
    # median = train['SalePrice'].median()
    # sample_incomplete['SalePrice'].fillna(median, inplace = True)
    
    
    num_attr = train.select_dtypes(exclude = 'object').columns.values.tolist()# object 형 제외 즉 수치형 특성의 column 값을 리스트로 반환
    cat_attr = train.select_dtypes(include = 'object').columns.values.tolist()# object 형 즉 범주형 특성의 column 값을 리스트로 반환
    templist = []
    
    #imputer 함수 사용법'
    #imputer = Imputer(strategy = "median")
    #num = train.drop(cat_attr, axis = 1)
    #imputer.fit(num)
    # statistics_ 에 저장 
    
    #OneHotEncoder 사용법
    #cat = train[cat_attr]
   # cat_encoded, cat_categories = cat.factorize()
   # encoder = OneHotEncoder()
   # cat_onehot = encoder.fit(cat_encoded.reshape(-1,1))
  
    
    for temp in cat_attr: # 만약 범주형 특성에서 Nan 값이 500개 이상이라면 해당 특성은 쓸모 없다 판단하여 제외시킨다.
        su = train[temp].isnull().sum() # NaN 값의 갯수 
        
        if su> 500: # 500개 이상일시 
            train = train.drop(temp, axis = 1) # 해당 특성 제외
            templist.append(temp)
    
    for temp in templist: # 삭제 시킨 특성을 범주형 특성 리스트에서도 제외시킵니다. 
        cat_attr.remove(temp)
    

    train=train.dropna(subset = cat_attr) # 범주형 특성 결측값 처리 
 
    ReTrain = train.drop("SalePrice", axis =1) # 범주형 특성 결측값 처리한 데이터 
    train_labels = train["SalePrice"].copy() # label 
    num_attr.remove('SalePrice')
    
    # 수치형 파이프라인 
    num_pipeline = Pipeline([
                            ('selector', DataFrameSelector(num_attr)),
                            ('imputer', Imputer(strategy="median")),
                            ('attribs_adder', CombinedAttributesAdder()),
                            ('std_scaler', StandardScaler()),
                            ])
    # 범주형 파이프라인 
    cat_pipeline = Pipeline([
                            ('selector', DataFrameSelector(cat_attr)),
                            ('cat_encoder', OneHotEncoder(sparse=False)),
                            ])
    # 수치형 + 범주형파이프라인 
    
    full_pipeline = FeatureUnion(transformer_list=[
            ("num_pipeline", num_pipeline),
            ("cat_pipeline", cat_pipeline),
        ])
 
    # 최종 정제된 데이터 
    train_prepared = full_pipeline.fit_transform(ReTrain)
   
    # 선형모델 rmse 실험 
    lin_reg = LinearRegression() # 선형모델 생성 
    lin_reg.fit(train_prepared,train_labels) # 학습 
    lin_prediction = lin_reg.predict(train_prepared) # 선형모델로 예측한 값 
    lin_mse = mean_squared_error(train_labels, lin_prediction) # 예측한 값과 레이블의 차이
    lin_rmse = np.sqrt(lin_mse) # 차이에 root 
    print("선형모델 rmse : ",lin_rmse)
    
    # 결정트리 rmse 실험 
    tre_reg = DecisionTreeRegressor() # 결정트리모델 생성 
    tre_reg.fit(train_prepared,train_labels) # 학습 
    tre_prediction = tre_reg.predict(train_prepared) # 결정트리로 예측한 값 
    tre_mse = mean_squared_error(train_labels, tre_prediction) # 예측한 값과 레이블의 차이 
    tre_rmse = np.sqrt(tre_mse) 
    print("결정트리 rmse : ",tre_rmse)
    
    # 랜덤포레스트모델 (앙상블 기법)
    fo_reg = RandomForestRegressor() # 모델 생성 
    fo_reg.fit(train_prepared, train_labels) # 모델 학습 
    fo_prediction = fo_reg.predict(train_prepared) # 모델로 예측값 도출
    fo_mse = mean_squared_error(train_labels, fo_prediction) # mse 값 
    fo_rmse = np.sqrt(tre_mse) # rmse 값 
    print("랜덤모델 rmse : ",fo_rmse)
    
    
    # 교차검증을 사용한 평가. 
    # mse 가 아닌 nmse 를 사용 
    # 선형모델의 경우 2, 결정트리,랜덤포레스트 10 의 cv 를 사용 
    # 선형모델의 경우 교차검증 사용시 큰 값의 숫자가 출력되는데 이유는 찾지 못했습니다. 
    
    scores = cross_val_score(lin_reg, train_prepared, train_labels,scoring = "neg_mean_squared_error", cv = 2)
    lin_rmse_score = np.sqrt(-scores)
    display_score(lin_rmse_score)
    
    scores = cross_val_score(tre_reg,train_prepared, train_labels,scoring = "neg_mean_squared_error", cv = 10)
    tre_rmse_score = np.sqrt(-scores)
    display_score(tre_rmse_score)
    
    scores = cross_val_score(fo_reg,train_prepared, train_labels,scoring = "neg_mean_squared_error", cv = 10)
    fo_rmse_score = np.sqrt(-scores)
    display_score(fo_rmse_score)
    
    
    #모델이 정해진 후에 그에따른 가장 최적의 파라미터를 찾는다.
    
    # 그리드 탐색 총 90번의 탐색 진행 
    param_grid = [
        {'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
        {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]},    
    ]
    
    grid_search = GridSearchCV(fo_reg, param_grid, cv = 5, scoring = 'neg_mean_squared_error',
                                return_train_score = True)
    grid_search.fit(train_prepared, train_labels)
    
    #랜덤 탐색 랜덤으로 탐색 진행 (크기가 클수록 효과적)
   
    param_dis = {
        'n_estimators':randint(low =1, high = 200),
        'max_features':randint(low = 1, high = 8),
    }
    
    rnd_search = RandomizedSearchCV(fo_reg,param_distributions = param_dis,
                                    n_iter = 10, cv = 5, scoring = 'neg_mean_squared_error',
                                    random_state = 42, n_jobs = -1)
    
    rnd_search.fit(train_prepared, train_labels)
    
    
    cvres = grid_search.cv_results_ # 그리드 탐색의 결과 
        
    print('-------------- 그리드 탐색 ---------------')
    for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]): # score 와 params 출력
        print(np.sqrt(-mean_score),params)
        

    cvres = rnd_search.cv_results_ # 랜덤 탐색의 결과
    
    print('')
    print('-------------- 랜덤 탐색 ---------------')
        
    for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]): # score 와 params 출력
        print(np.sqrt(-mean_score),params)
    
    
    final_model = rnd_search.best_estimator_ # ppt 는 grid search로 찾은 파라미터를 사용하였지만
    # 저는 random search 로 찾은 파라미터를 사용하였습니다. 
    
 
    
    test = test_set.copy() # 최종 test set 
    test['SalePrice'] = sample_set['SalePrice'] # test set 에 정답 레이블을 매칭시켜준다.
    
    
    num_attr = test.select_dtypes(exclude = 'object').columns.values.tolist() # 수치형 특성들만 리스트로 반환
    cat_attr = test.select_dtypes(include = 'object').columns.values.tolist() # 범부형 특성들만 리스트로 반환
    templist = []
    IDlist = []
    
    for temp in cat_attr:
        su = test[temp].isnull().sum() # 결측값의 합 
        
        if su> 500: # 결측값의 값이 500 이상 
            test = test.drop(temp, axis = 1) # 이면 쓸모 없는 특성이라 생각하고 특성  drop
            templist.append(temp)
    
    for temp in templist:
        cat_attr.remove(temp) # drop 된 특성들을 범주형 특성리스트에서도 지운다. 
        
    
        
    test=test.dropna(subset = cat_attr) # 마지막으로 범주형 특성중에 결측값이 있는 샘플 제거 
   
    test_labels = test['SalePrice'].copy() # 정답 레이블 분리 
    test = test.drop('SalePrice',axis = 1)
    
    print(cat_attr)
    # train_set 에서 데이터 정제를 하던 중 없어지는 특성이 있어 인코딩 할때 문제가 발생되는데
    # train.csv 파일에 다른 내용은 같고 없
    test_prepared = full_pipeline.transform(test) # 데이터 정제 
   
    final_predictions = final_model.predict(test_prepared) #최종 모델로 예측 
    
    final_mse = mean_squared_error(test_labels, final_predictions)
    final_rmse = np.sqrt(final_mse)
    
    print("최종 rmse", final_rmse) # 최종 rmse 값 일반적으로 교차검증을 사용할때보다 크게나옴 
    

<Figure size 640x480 with 1 Axes>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

<Figure size 2000x1500 with 42 Axes>

SalePrice          1.000000
OverallQual        0.790982
TotalSquareFeet    0.716883
GrLivArea          0.708624
GarageCars         0.640409
GarageArea         0.623431
TotalBsmtSF        0.613581
TotalBathRoom      0.613005
1stFlrSF           0.605852
FullBath           0.560664
TotRmsAbvGrd       0.533723
YearBuilt          0.522897
YearRemodAdd       0.507101
GarageYrBlt        0.486362
MasVnrArea         0.477493
Fireplaces         0.466929
BsmtFinSF1         0.386420
LotFrontage        0.351799
WoodDeckSF         0.324413
2ndFlrSF           0.319334
OpenPorchSF        0.315856
HalfBath           0.284108
LotArea            0.263843
BsmtFullBath       0.227122
BsmtUnfSF          0.214479
BedroomAbvGr       0.168213
ScreenPorch        0.111447
PoolArea           0.092404
MoSold             0.046432
3SsnPorch          0.044584
BsmtFinSF2        -0.011378
BsmtHalfBath      -0.016844
MiscVal           -0.021190
Id                -0.021917
LowQualFinSF      -0.025606
YrSold            -0

<Figure size 1200x800 with 16 Axes>

선형모델 rmse :  21087.397569339468
결정트리 rmse :  0.0
랜덤모델 rmse :  0.0
점수 :  [1.77229615e+14 1.83018049e+16]
평균 :  9239517254676160.0
표준편차 :  9062287639529172.0
점수 :  [34083.13451815 41611.69147796 33347.61322726 45101.2675497
 43681.49639696 31691.00367235 53397.1460975  39518.44400677
 67319.84679378 37170.89254149]
평균 :  42692.253628192935
표준편차 :  10257.133440814066
점수 :  [26392.46092544 30450.83420102 24700.82727964 41095.91271116
 33048.77237039 30192.40954203 25071.58678264 25938.04143325
 45142.02147341 31156.08445611]
평균 :  31318.895117507425
표준편차 :  6540.3470929589685
-------------- 그리드 탐색 ---------------
46024.70438286895 {'max_features': 2, 'n_estimators': 3}
39936.248710238004 {'max_features': 2, 'n_estimators': 10}
37208.93254051117 {'max_features': 2, 'n_estimators': 30}
45275.10692256958 {'max_features': 4, 'n_estimators': 3}
39677.12146294162 {'max_features': 4, 'n_estimators': 10}
35433.89484002315 {'max_features': 4, 'n_estimators': 30}
42644.101962171815 {'max_features': 

ValueError: Found unknown categories ['Po'] in column 18 during transform